### Set standard imports

In [ ]:
import matplotlib,lmfit,os,sys
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk
#for more comfort use
#%matplotlib qt
import KiMoPack.plot_func as pf
from importlib import reload
#reload(pf)

### Load Data and perform standard corrections

In [ ]:
ta=pf.TA('sample_1.hdf5',path='Data')  #reload saved projects using the same syntax, this reloads the parameter too

### Perform deeper fitting using external fitting function

In [ ]:
import function_library as func                      # load the file with the example functions. must be in the active folder
reload(func)                                              # usefull if you play with the function

In [ ]:
plt.close('all')
ta.mod=func.P12                            # if you want to use function P12 from the standard function library
#ta.mod=func.consec_oscil                  # if you want to fit consecutive and osciallations
par=lmfit.Parameters()
par.add('k0',value=1/1.6,vary=True)        # this function has a lot of parameter, they can be named
par.add('k1',value=1/0.4,vary=True)        # but a rate must be starting with "k" (this is how it is regognized)
par.add('k2',value=1/6e11,vary=True)
#par.add('f0',value=5)                     # add osciallation with this frequency
#par.add('tk0',value=5)                    # (optional) add decay of osciallation

par.add('t0',value=0.025,min=-2,max=2,vary=False)          # if applicable, keep those
par.add('resolution',value=0.25,min=0.04,max=0.5,vary=False)
#par.add('infinite')
#par.add('background')
#par.add('explicit_GS')                                   # add explicit background
ta.par=par

ta.Fit_Global()                                           # Fitting syntax stays the same

In [ ]:
ta.Plot_fit_output()

In [ ]:
ta.Save_Powerpoint()

In [ ]:
ta.par=ta.fit_par #reuse the fitted parameter
ta.Fit_Global(dump_paras=True)
ta.Fit_Global(dump_shapes=True)

In [ ]:
ds=pandas.read_csv('spectra.csv',index_col=0)
ds.columns=['GS']
ta.Fit_Global(ext_spectra=ds)

ta.par.add('ext_spectra_shift',value=0,vary=True)
ta.par.add('ext_spectra_scale',value=0,vary=True)

ta.par.add('ext_spectra_guide',value=0,vary=True)

### Optimise Chirp

In [ ]:
ta1=ta.Copy() # before chirp fitting I recommend to 
ta1.Fit_Global(fit_chirp=True)

### Error Analysis

In [ ]:
ta.Fit_Global(confidence_level=0.95) 

### Fitting multiple datasets

In [ ]:
ta1=pf.TA(filename='sample_2.hdf5',path='Data')          # lets load a single second project measured at a different power
other_projects=[ta1]

other_projects=[pf.TA(filename,path='Data') for filename in ['sample_2.hdf5','sample_3.hdf5']]

other_projects=pf.GUI_open()

In [ ]:
ta.Fit_Global(multi_project=other_projects)#fit both projects together,
ta.Fit_Global(multi_project=other_projects,unique_parameter=['f0']) #unique parameter allows to additionally freeze a parameter (that e.g. were optimized b)
ta.Fit_Global(multi_project=other_projects,unique_parameter=['f0'],weights=[1,2])
ta.Fit_Global(multi_project=other_projects,same_DAS = True)

### Using Global minimum search

In [ ]:
plt.close('all')
ta.mod=func.P12             # if you want to use function P12 from the standard function library
par=lmfit.Parameters()
par.add('k0',value=1/1.6,vary=True,min=1/4,max=1) 
par.add('k1',value=1/0.4,vary=True,min=1/4,max=1)          
par.add('k2',value=1/6e11,vary=True,min=1/1e13,max=1)
par.add('t0',value=0.025,min=-2,max=2,vary=False)
par.add('resolution',value=0.25,min=0.04,max=0.5,vary=False)
#par.add('background')
#par.add('infinite')
ta.par=par
ta.Fit_Global(use_ampgo=True)                                           # Fitting syntax stays the same
'''Sometimes this optimization runs into extremely small value that overflow the double precission 
(making a lot of warnings). As a temporary fix enable the following line that will turn of all warnings.
as most likely these solutions will be rejected anyhow, this should not be a big problem. Alternative is to 
give more restrictive limits.'''
#np.seterr(all="ignore")

### export the results as images and save the project

In [ ]:
ta.Save_project()         # This saves the whole project including raw data, fits and chirp correction into a single file

In [ ]:
ta.Save_Plots(savetype='png')         # This saves all the figures and a overview powerpoint savetype=['png','svg']

### Sometimes required options applicable also to fit plotting

In [ ]:
ta.Man_Chirp()                   # used to redo the chirp-correction
ta.Save_data()                   # used to dump the Data and fits to disk

ta.cmap=pf.cm.viridis            # use different colour scheme (can also be a list of colours)
pf.changefonts(weight='bold',font='standard',SMALL_SIZE=11,MEDIUM_SIZE=13,LARGE_SIZE=18)       